In [3]:
import zfit
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit
import warnings
import os
from itertools import combinations
import pandas as pd
from PDFs import (
    FullAngular_Physical_PDF, 
    FullAngular_Transformed_PDF, 
    get_inverse_values, 
    apply_transformation_equations,
    get_physical_region_scan
)
import matplotlib.pyplot as plt
import cmsstyle
import sys
from plot_tools import create_axes_for_pulls, plot_model 
import plot_tools 
import customPDFs
import mass_models
import tensorflow as tf
from PIL import Image
import seaborn as sns
import mplhep as hep
import os, sys
import common_tools
from scipy.stats import gaussian_kde
from matplotlib.gridspec import GridSpec
from scipy import stats
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use("CMS")

/home/ghcp/miniconda3/envs/haza_wokr_env/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warning, set ZFIT_DISABLE_TF_WARNINGS=1.
  warnings.warn(


Welcome to JupyROOT 6.30/04


/home/ghcp/miniconda3/envs/haza_wokr_env/lib/python3.8/site-packages/zfit/core/basemodel.py:199: UserWarning: For the future, also decorate _pdf with @supports and specify what you support (such as 'norm=True' to keep the same behavior as before)
  warnings.warn(


# Funciones importante

In [4]:
zfit.settings.set_seed(42)
np.random.seed(42)
warnings.simplefilter('ignore')


def calculate_jacobian_numerical(func, params, keys_out, epsilon=1e-5):
    """Calcula la matriz Jacobiana numéricamente."""
    n_params = len(params)
    n_out = len(keys_out)
    J = np.zeros((n_out, n_params))
    
    def func_wrapper(p_args):
        res_dict = func(*p_args)
        return np.array([res_dict[k] for k in keys_out])

    for i in range(n_params):
        p_plus = np.copy(params)
        p_minus = np.copy(params)
        p_plus[i] += epsilon
        p_minus[i] -= epsilon
        
        f_plus = func_wrapper(p_plus)
        f_minus = func_wrapper(p_minus)
        
        deriv = (f_plus - f_minus) / (2 * epsilon)
        J[:, i] = deriv
    return J


# Generación de MC

In [5]:

#MAIN function 
folder_trans = "Plots/Transformed_Space_vFinal"
folder_phys_zoom = "Plots/Physical_Space_Zoom_vFinal"
folder_phys_full = "Plots/Physical_Space_Full_vFinal"

for f in [folder_trans, folder_phys_zoom, folder_phys_full]:
    os.makedirs(f, exist_ok=True)

print(f">>> Directorios listos:\n    - {folder_trans}\n    - {folder_phys_zoom}\n    - {folder_phys_full}")
# GENERACIÓN (toys) ---
obs = zfit.Space('cosThetaL', limits=(-1, 1)) * zfit.Space('cosThetaK', limits=(-1, 1)) * zfit.Space('phi', limits=(-np.pi, np.pi))

# Valores verdaderos físicos lhcb
true_vals_phys = [0.684, 0.014, 0.029, 0.050, -0.145, -0.136, -0.204, 0.077]
phys_keys = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
true_dict = dict(zip(phys_keys, true_vals_phys))

print(">>> Generando datos (Toy MC)...")
pdf_gen = FullAngular_Physical_PDF(obs, *true_vals_phys)
sampler = pdf_gen.create_sampler(n=2000) 
sampler.resample()


>>> Directorios listos:
    - Plots/Transformed_Space_vFinal
    - Plots/Physical_Space_Zoom_vFinal
    - Plots/Physical_Space_Full_vFinal
>>> Generando datos (Toy MC)...


Fit in transformed space

In [6]:

raw_init = get_inverse_values(true_vals_phys)
r_keys = ['rFL', 'rS3', 'rS9', 'rAFB', 'rS4', 'rS7', 'rS5', 'rS8']
param_names_fit = [f"{k}_fit" for k in r_keys]

true_vals_trans_dict = dict(zip(param_names_fit, raw_init))

params = {k: zfit.Parameter(p_name, v, step_size=0.01) 
            for k, p_name, v in zip(r_keys, param_names_fit, raw_init)}

pdf_fit = FullAngular_Transformed_PDF(obs, params['rFL'], params['rS3'], params['rS9'], params['rAFB'],params['rS4'], params['rS7'], params['rS5'], params['rS8'])

print(">>> 2. Ejecutando Minimización (Minuit)...")
nll = zfit.loss.UnbinnedNLL(model=pdf_fit, data=sampler)
minimizer = zfit.minimize.Minuit(tol=0.01) 
result = minimizer.minimize(nll)
m = result.info['minuit'] 

print(">>> 3. Calculando Errores MINOS...")
m.minos() 

# =======================================
# ESPACIO TRANSFORMADO (VALORES DE MINUIT)
# =======================================
print("\n" + "="*80)
print(">>> Errores MINOS (Espacio Transformado)")
print("="*80)
print(f"{'PARAM':<10} | {'VALOR':<10} | {'ERROR -':<10} | {'ERROR +':<10} | {'VERDAD':<10}")
print("-" * 80)

# Aquí guarda los valores CORRECTOS de Minuit para usarlos después
minuit_best_values = []

for rk, pname in zip(r_keys, param_names_fit):
    val = m.values[pname]
    minuit_best_values.append(val) # guarda el valor exacto de Minuit
    
    err_low = m.merrors[pname].lower  
    err_high = m.merrors[pname].upper 
    truth = true_vals_trans_dict[pname]
    
    print(f"{rk:<10} | {val:.4f}     | {err_low:.4f}     | +{err_high:.4f}    | {truth:.4f}")
print("="*80 + "\n")

# ===========================================
# ESPACIO FÍSICO (CORREGIDO PARA USAR MINUIT)
# ===========================================
print("\n" + "="*80)
print(">>> Errores Gaussianos (Matriz de Covarianza Propagada)")
print("="*80)

# recupera la matriz de covarianza
param_objs_ordered = [params[k] for k in r_keys]
cov_trans = result.covariance(params=param_objs_ordered)

# IMPORTANTE usar los valores de MINUIT, no los de zfit.Parameter (revisar pq)
best_fit_r_values = np.array(minuit_best_values)

# Jacobiano y Propagación
J = calculate_jacobian_numerical(apply_transformation_equations, best_fit_r_values, phys_keys)
cov_phys = J @ cov_trans @ J.T
phys_errors_sigma = np.sqrt(np.diag(cov_phys))
phys_errors_dict = dict(zip(phys_keys, phys_errors_sigma))

# 4. Tabla (Calculada con los valores de Minuit)
print(f"{'PARAM':<10} | {'VALOR':<10} | {'ERROR (+/-)':<15} | {'VERDAD':<10}")
print("-" * 80)

best_fit_phys_dict = apply_transformation_equations(*best_fit_r_values)

for k in phys_keys:
    val = best_fit_phys_dict[k]
    err = phys_errors_dict[k]
    tru = true_dict[k]
    print(f"{k:<10} | {val:.4f}     | +/- {err:.4f}      | {tru:.4f}")
print("="*80 + "\n")


>>> 2. Ejecutando Minimización (Minuit)...
>>> 3. Calculando Errores MINOS...

>>> Errores MINOS (Espacio Transformado)
PARAM      | VALOR      | ERROR -    | ERROR +    | VERDAD    
--------------------------------------------------------------------------------
rFL        | 0.3667     | -0.0334     | +0.0346    | 0.3861
rS3        | 0.0596     | -0.1200     | +0.1216    | 0.0888
rS9        | 0.1201     | -0.1300     | +0.1391    | 0.1864
rAFB       | 0.0773     | -0.0590     | +0.0571    | 0.2189
rS4        | -0.5490     | -0.1600     | +0.1373    | -0.7812
rS7        | -0.3659     | -0.0888     | +0.0734    | -0.4293
rS5        | -0.3709     | -0.0594     | +0.0553    | -0.4484
rS8        | 0.3803     | -0.1192     | +0.1356    | 0.3654


>>> Errores Gaussianos (Matriz de Covarianza Propagada)
PARAM      | VALOR      | ERROR (+/-)     | VERDAD    
--------------------------------------------------------------------------------
FL         | 0.6755     | +/- 0.0149      | 0.6840
S3   

# Contornos 

In [5]:


#GENERACIÓN DEL MAPA FÍSICO ---
df_phys_region = get_physical_region_scan(n_points=1000)
indices = list(range(8))
pairs_indices = list(combinations(indices, 2))
total_plots = len(pairs_indices)

for i, (idx_x, idx_y) in enumerate(pairs_indices):
    
    rx, ry = param_names_fit[idx_x], param_names_fit[idx_y]
    px, py = phys_keys[idx_x], phys_keys[idx_y]

    print(f"    [{i+1}/{total_plots}] {px} vs {py} ...", end="\r")

    # 1. OBTENER CONTORNO (Espacio R)
    contour_r = m.mncontour(rx, ry, cl=0.3935, size=50)

    # ---------------------------------------------------------
    # PLOT A: Espacio Transformado
    # ---------------------------------------------------------
    plt.figure(figsize=(8, 7))
    plt.plot(contour_r[:, 0], contour_r[:, 1], 'b-', linewidth=2, label='1$\sigma$ Contour')
    plt.plot(m.values[rx], m.values[ry], 'bo', label='Best Fit', zorder=10)
    true_tx = true_vals_trans_dict[rx]
    true_ty = true_vals_trans_dict[ry]
    plt.plot(true_tx, true_ty, 'r*', markersize=14, markeredgecolor='k', label='True Value', zorder=15)

    # MINOS Errors visuales
    val_x = m.values[rx]; val_y = m.values[ry]
    minos_x_low = val_x + m.merrors[rx].lower; minos_x_high = val_x + m.merrors[rx].upper
    minos_y_low = val_y + m.merrors[ry].lower; minos_y_high = val_y + m.merrors[ry].upper
    
    plt.axvline(minos_x_low, color='k', linestyle='--', alpha=0.4)
    plt.axvline(minos_x_high, color='k', linestyle='--', alpha=0.4)
    plt.axhline(minos_y_low, color='k', linestyle='--', alpha=0.4)
    plt.axhline(minos_y_high, color='k', linestyle='--', alpha=0.4, label='MINOS Errors')
    
    width_x = minos_x_high - minos_x_low; width_y = minos_y_high - minos_y_low
    margin = 0.4
    plt.xlim(minos_x_low - width_x * margin, minos_x_high + width_x * margin)
    plt.ylim(minos_y_low - width_y * margin, minos_y_high + width_y * margin)

    plt.xlabel(rx); plt.ylabel(ry)
    plt.title(f"Transformed: {rx} vs {ry}")
    plt.grid(True, alpha=0.3); plt.legend()
    plt.savefig(f"{folder_trans}/Trans_{rx}_vs_{ry}.png")
    plt.close()

    # -------------
    # PLOTS FÍSICOS
    # -------------
    n_pts = len(contour_r)
    r_matrix = np.tile(best_fit_r_values, (n_pts, 1))
    r_matrix[:, idx_x] = contour_r[:, 0]
    r_matrix[:, idx_y] = contour_r[:, 1]
    trans_contour_dict = apply_transformation_equations(r_matrix[:, 0], r_matrix[:, 1], r_matrix[:, 2], r_matrix[:, 3],r_matrix[:, 4], r_matrix[:, 5], r_matrix[:, 6], r_matrix[:, 7])
    cx_phys = trans_contour_dict[px]
    cy_phys = trans_contour_dict[py]

    # CÁLCULO DE ERRORES GEOMÉTRICOS
    x_min_cont = np.min(cx_phys)
    x_max_cont = np.max(cx_phys)
    y_min_cont = np.min(cy_phys)
    y_max_cont = np.max(cy_phys)
    
    bf_x = best_fit_phys_dict[px]
    bf_y = best_fit_phys_dict[py]
    
    err_x_up = x_max_cont - bf_x
    err_x_down = bf_x - x_min_cont
    err_y_up = y_max_cont - bf_y
    err_y_down = bf_y - y_min_cont

    def plot_physical(view_mode, x_lims=None, y_lims=None):
        plt.figure(figsize=(9, 8))
        alpha_cloud = 0.15 if view_mode == 'Full' else 0.05
        
        # plt.scatter(df_phys_region[px], df_phys_region[py],c='gray', s=1, alpha=alpha_cloud, label='Allowed Region', zorder=0)
        
        plt.plot(cx_phys, cy_phys, 'g-', linewidth=2.5, label='1$\sigma$ Contour')
        plt.plot(bf_x, bf_y, 'bo', markersize=6, label='Best Fit', zorder=10)
        plt.plot(true_dict[px], true_dict[py], 'r*', markersize=14, markeredgecolor='k', label='True Value', zorder=11)
        
        plt.axvline(x_min_cont, color='red', linestyle='--', linewidth=1, alpha=0.7)
        plt.axvline(x_max_cont, color='red', linestyle='--', linewidth=1, alpha=0.7)
        plt.axhline(y_min_cont, color='red', linestyle='--', linewidth=1, alpha=0.7)
        plt.axhline(y_max_cont, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Contour Limits')

        plt.xlabel(px, fontsize=14); plt.ylabel(py, fontsize=14)
        
        if view_mode == 'Zoom':
            title_str = (f"{px} = {bf_x:.4f} (+{err_x_up:.4f}/-{err_x_down:.4f})\n"
                            f"{py} = {bf_y:.4f} (+{err_y_up:.4f}/-{err_y_down:.4f})")
            plt.title(title_str, fontsize=11, color='blue')
            if x_lims: plt.xlim(x_lims); 
            if y_lims: plt.ylim(y_lims)
        else:
            plt.title(f"FULL: {px} vs {py}", fontsize=12)
            plt.xlim(df_phys_region[px].min(), df_phys_region[px].max())
            plt.ylim(df_phys_region[py].min(), df_phys_region[py].max())

        plt.grid(True, alpha=0.3)
        plt.legend(loc='upper right', frameon=True, fontsize=9)
        
        folder = folder_phys_zoom if view_mode == 'Zoom' else folder_phys_full
        plt.savefig(f"{folder}/Phys_{px}_vs_{py}_{view_mode}.png", dpi=100)
        plt.close()

    # Definir límites de zoom
    mx = (x_max_cont - x_min_cont) * 0.4
    my = (y_max_cont - y_min_cont) * 0.4
    
    plot_physical('Zoom', x_lims=(x_min_cont-mx, x_max_cont+mx), y_lims=(y_min_cont-my, y_max_cont+my))
    plot_physical('Full')


# Proyecciones v1

In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib.gridspec import GridSpec
from scipy.stats import chi2 as _chi2
from scipy.integrate import quad


def analytic_proj_cosK(x, FL):
    """
    Densidad normalizada en [-1,1] para cosThetaK (fallback).
    """
    x = np.asarray(x)
    vals = 0.75 * (1.0 - FL) * (1.0 - x**2) + 1.5 * FL * x**2
    # normalizar numéricamente
    norm = np.trapz(vals, x) if x.size > 1 else None
    return vals if norm is None else vals / (norm + 1e-12)

def analytic_proj_cosL(x, FL, AFB):
    """
    Densidad normalizada en [-1,1] para cosThetaL (fallback).
    f(x) = 3/8 (1+FL) + 3/8 (1-3FL) x^2 + AFB * x
    """
    x = np.asarray(x)
    vals = (3.0/8.0) * (1.0 + FL) + (3.0/8.0) * (1.0 - 3.0 * FL) * x**2 + AFB * x
    # normalizamos numéricamente si es vector
    if x.size > 1:
        norm = np.trapz(vals, x)
        return vals / (norm + 1e-12)
    return vals

def analytic_proj_phi(phi, S3=0.0, S9=0.0, FL=None):
    """
    Fallback para phi. Si no conocemos la forma exacta, usamos la uniforme 1/(2π)
    y añadimos modulaciones cos(2φ), sin(2φ) con amplitudes S3,S9 escaladas de forma conservadora.
    """
    phi = np.asarray(phi)
    base = np.ones_like(phi) / (2.0 * np.pi)
    # mod = (4.0/3.0) * (S3 * np.cos(2.0 * phi) + S9 * np.sin(2.0 * phi))
    # vals = base * (1.0 + mod)
    mod = S3 * np.cos(2.0 * phi) + S9 * np.sin(2.0 * phi)
    vals = base * (1.0 + mod)
    if phi.size > 1:
        norm = np.trapz(vals, phi)
        return vals / (norm + 1e-12)
    return vals

hep.style.use("CMS")

def _sampler_to_numpy(sampler):
    arr = np.asarray(sampler)
    if arr.ndim == 2 and arr.shape[1] == 3:
        return arr


    
    if hasattr(sampler, "numpy"):
        arr = sampler.numpy()
        arr = np.asarray(arr)
        if arr.ndim == 2 and arr.shape[1] == 3:
            return arr
        # a veces sampler.numpy() devuelve (3,N)
        if arr.ndim == 2 and arr.shape[0] == 3:
            return arr.T


    if hasattr(sampler, "resample"):
        out = sampler.resample()
        arr = np.asarray(out)
        if arr.ndim == 2 and arr.shape[1] == 3:
            return arr
    if hasattr(sampler, "sample"):
        out = sampler.sample()
        arr = np.asarray(out)
        if arr.ndim == 2 and arr.shape[1] == 3:
            return arr


def _integrate_pdf_on_bin(func, a, b, args=(), n_quad_points=50):
    val, err = quad(lambda xx: func(xx, *args), a, b, epsabs=1e-8, epsrel=1e-6, limit=100)
    return val

def _rebin_until_expected_ok(edges, counts, expected, min_expected):
    """
    Rebin simple: iteramos bins de izquierda a derecha acumulando hasta que expected_acc >= min_expected,
    luego cerramos un bin combinado y seguimos.
    Devuelve new_edges, new_counts, new_expected.
    """
    new_edges = []
    new_counts = []
    new_expected = []

    cur_left = edges[0]
    cur_counts = 0.0
    cur_expected = 0.0

    for i in range(len(expected)):
        cur_right = edges[i+1]
        cur_counts += counts[i]
        cur_expected += expected[i]

        if cur_expected >= min_expected or i == len(expected) - 1:
            new_edges.append(cur_left)
            new_edges.append(cur_right)
            new_counts.append(cur_counts)
            new_expected.append(cur_expected)
            # reset
            if i != len(expected) - 1:
                cur_left = cur_right
                cur_counts = 0.0
                cur_expected = 0.0

    # transformar new_edges a array de edges ordenadas (puede haber duplicados)
    # new_edges actualmente [left0,right0,left1,right1,...], convertimos a monotónica
    e = [new_edges[0]]
    for i in range(0, len(new_edges), 2):
        right_i = new_edges[i+1]
        e.append(right_i)
    e = np.array(e)
    return e, np.array(new_counts), np.array(new_expected)

def plot_analytical_projections(sampler, phys_params_dict, folder_out, n_bins=40, min_expected=5.0, rebin_if_needed=True):
    """
    Genera proyecciones 1D (cosThetaK, cosThetaL, phi) comparando toy (sampler) con
    la PDF analítica construida a partir de phys_params_dict.
    Guarda PNGs en folder_out y devuelve un diccionario con estadísticas de ajuste.
    - sampler: objeto que contiene los eventos (columnas: cosL, cosK, phi)
    - phys_params_dict: dict con claves mínimo: 'FL','AFB','S3','S9' (otras son ignoradas)
    """
    os.makedirs(folder_out, exist_ok=True)

    data = _sampler_to_numpy(sampler)
    # asumimos columnas [cosL, cosK, phi]
    cosL = data[:, 0]
    cosK = data[:, 1]
    phi  = data[:, 2]
    n_events = len(cosL)

    # parametros (fallback a 0 si no están)
    FL  = float(phys_params_dict.get('FL', 0.0))
    AFB = float(phys_params_dict.get('AFB', 0.0))
    S3  = float(phys_params_dict.get('S3', 0.0))
    S9  = float(phys_params_dict.get('S9', 0.0))

    # seleccion de funciones analíticas (usar PDFs.py si está)
    f_cosK = lambda x: analytic_proj_cosK(x, FL)
    f_cosL = lambda x: analytic_proj_cosL(x, FL, AFB)
    f_phi  = lambda x: analytic_proj_phi(x, S3, S9, FL)


    results = {}

    vars_cfg = [
        (cosK, f_cosK, r'$\cos\theta_K$', 'CosThetaK', (-1.0, 1.0), 1),   # params_effective=1 (FL)
        (cosL, f_cosL, r'$\cos\theta_L$', 'CosThetaL', (-1.0, 1.0), 2),   # params_effective=2 (FL, AFB)
        (phi,  f_phi,  r'$\phi$',         'Phi',       (-np.pi, np.pi), 2) # params_effective approx=2 (S3,S9)
    ]

    for arr, func, xlabel, name, limits, params_effective in vars_cfg:
        edges = np.linspace(limits[0], limits[1], n_bins + 1)
        counts, _ = np.histogram(arr, bins=edges)
        centers = 0.5 * (edges[:-1] + edges[1:])
        bin_widths = edges[1:] - edges[:-1]

        expected = np.zeros_like(counts, dtype=float)
        for i in range(len(expected)):
            a, b = edges[i], edges[i+1]
            # si es phi y el bin cruza el -pi/pi, ajustamos para integrar correctamente (no necesario si edges exactos)
            expected[i] = _integrate_pdf_on_bin(func, a, b) * n_events

        # rebin si hay bins esperados pequeños
        if rebin_if_needed and np.any(expected < min_expected):
            edges_new, counts_new, expected_new = _rebin_until_expected_ok(edges, counts, expected, min_expected)
            # recompute centers and widths
            edges = edges_new
            counts = counts_new
            expected = expected_new
            centers = 0.5 * (edges[:-1] + edges[1:])
            bin_widths = edges[1:] - edges[:-1]

        # evitar ceros exactos
        eps = 1e-9
        expected_safe = np.where(expected <= 0, eps, expected)

        # pulls y chi2 (usar E en denominador - prueba de Pearson)
        pulls = (counts - expected) / np.sqrt(expected_safe)
        mask_valid = expected > 0  # bins utilizados
        chi2_val = np.sum(((counts[mask_valid] - expected[mask_valid])**2) / expected_safe[mask_valid])

        n_bins_used = mask_valid.sum()
        ndof = max(1, int(n_bins_used) - int(params_effective))
        p_val = _chi2.sf(chi2_val, ndof)
        chi2_red = chi2_val / ndof

        # para la curva continua (conteos por bin) evaluamos f(x) en grilla fina y convertimos a counts/bin
        x_fine = np.linspace(limits[0], limits[1], 2000)
        f_fine = func(x_fine)
        # asegurar que f_fine está normalizada a densidad (integral 1) — si func ya devuelve densidad esto no cambia
        integral_f = np.trapz(f_fine, x_fine)
        if integral_f <= 0:
            integral_f = 1.0
        f_fine_norm = f_fine / integral_f
        y_smooth_counts = f_fine_norm * n_events * ( (edges[1] - edges[0]) ) 




        #======
        # plot
        #======
        fig = plt.figure(figsize=(9, 9))
        gs = GridSpec(2, 1, height_ratios=[3.5, 1], hspace=0.08)
        ax0 = fig.add_subplot(gs[0])
        ax1 = fig.add_subplot(gs[1])

        # datos: puntos con error (sqrt(O))
        y_err_data = np.sqrt(counts)
        ax0.errorbar(centers, counts, yerr=y_err_data, xerr=bin_widths/2.0, fmt='ks', markersize=4, elinewidth=1, capsize=2, label='Toy (data)')

        # curva continua (suponemos bins uniformes iniciales para escala visual). Si rebin cambió bin widths, mostramos también los puntos esperados por bin
        ax0.plot(x_fine, y_smooth_counts, '-', linewidth=1, label='Analytical PDF', color='b')
        max_data = np.max(counts + y_err_data) # Altura del bin más alto + su barra de error
        max_model = np.max(y_smooth_counts)    # Altura máxima de la curva azul
        y_max_plot = max(max_data, max_model)
        # 3. Añadimos un margen del 40% (multiplicar por 1.4) para que quepa el cuadro de texto
        ax0.set_ylim(0, y_max_plot * 1.5) 

        ax0.set_ylabel(f'Events / bin', fontsize=14)
        ax0.set_xlim(limits)
        ax0.set_xticklabels([])
        ax0.set_ylabel(f'Events / bin', fontsize=14)
        ax0.set_xlim(limits)
        ax0.set_xticklabels([])
        ax0.grid(True, alpha=0.2)
        ax0.legend(loc='best', fontsize=12, frameon=False)

        hep.cms.label(data=False, loc=0, ax=ax0, rlabel="13 TeV")

        # anotación estadística
        param_text = (
            rf'$F_L = {FL:.3f}$' + '\n' +
            rf'$A_{{FB}} = {AFB:.3f}$' + '\n' +
            rf'$S_3 = {S3:.3f}$' + '\n' +
            rf'$S_9 = {S9:.3f}$' + '\n' +
            rf'$\chi^2/\mathrm{{ndof}} = {chi2_val:.2f}/{ndof} = {chi2_red:.2f}$' + '\n' +
            rf'$p$-value = {p_val:.3g}'
        )

        ax0.text(0.05, 0.95, param_text, transform=ax0.transAxes, fontsize=11, verticalalignment='top',bbox=dict( facecolor='white', alpha=0.01))

        # panel de pulls
        ax1.errorbar(centers, pulls, 
                     yerr=1.0,           # El error del pull normalizado es siempre 1
                     xerr=0,             # Generalmente no se pone error en X en los pulls para limpieza
                     fmt='ks',           # 'k'=negro, 'o'=círculo
                     markersize=1,       # Equivalente aproximado a s=22
                     elinewidth=1.0,     # Grosor de la barra vertical
                     capsize=0)          
        # Línea de referencia en 0
        ax1.axhline(0, color='black', linewidth=1.0, linestyle='-')
        ax1.axhline(0, color='black', linestyle='', linewidth=1, alpha=0.4)
        ax1.axhline(3, color='black', linestyle=':', linewidth=1, alpha=0.8, label=r'3$\sigma$')
        ax1.axhline(-3, color='black', linestyle=':', linewidth=1, alpha=0.8)
        ax1.set_xlabel(xlabel, fontsize=14)
        ax1.set_ylabel(r'Pull', fontsize=11)
        ax1.set_xlim(limits)
        ax1.set_ylim(-5.0, 5.0)
        ax1.grid(True, alpha=0.2)

        ax0.tick_params(axis='both', which='major', labelsize=14) 
        ax1.tick_params(axis='both', which='major', labelsize=14)
        # guardar
        outname = os.path.join(folder_out, f"CMS_Proj_{name}_Chi2.png")
        plt.savefig(outname, bbox_inches='tight', dpi=150)
        plt.close(fig)

        # guardar resultados
        results[name] = {
            'counts': counts,
            'edges': edges,
            'expected': expected,
            'pulls': pulls,
            'chi2': float(chi2_val),
            'ndof': int(ndof),
            'chi2_red': float(chi2_red),
            'p_value': float(p_val),
            'n_events': int(n_events),
            'params_used': params_effective,
            'plot_file': outname
        }

        print(f"[OK] {name}: χ2/ndof = {chi2_val:.2f}/{ndof} ({chi2_red:.2f}), p = {p_val:.3g}  — plot -> {outname}")

    return results


In [10]:

results_proj = plot_analytical_projections(
    sampler=sampler,
    phys_params_dict=best_fit_phys_dict,
    folder_out="Plots/Analytic_Projections",
    n_bins=30,           
    min_expected=5.0,   
    rebin_if_needed=True
)

print(">>> Proyecciones listas.")


[OK] CosThetaK: χ2/ndof = 22.36/29 (0.77), p = 0.805  — plot -> Plots/Analytic_Projections/CMS_Proj_CosThetaK_Chi2.png
[OK] CosThetaL: χ2/ndof = 20.82/28 (0.74), p = 0.833  — plot -> Plots/Analytic_Projections/CMS_Proj_CosThetaL_Chi2.png
[OK] Phi: χ2/ndof = 17.93/28 (0.64), p = 0.928  — plot -> Plots/Analytic_Projections/CMS_Proj_Phi_Chi2.png
>>> Proyecciones listas.


# Allowed region

In [ ]:
#GENERACIÓN DEL MAPA FÍSICO ---
df_phys_region = get_physical_region_scan(n_points=10000)
indices = list(range(8))
pairs_indices = list(combinations(indices, 2))
total_plots = len(pairs_indices)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds
import pandas as pd
from itertools import combinations
import math
import time

def constraints(params):
    """
    Mapeo explícito de las 5 Desigualdades Físicas.
    params = [FL, S3, S9, AFB, S4, S7, S5, S8]
    """
    FL, S3, S9, AFB, S4, S7, S5, S8 = params
    
    # --- Desigualdad 1: 0 <= FL <= 1 ---
    # Nota: Aunque esta se suele manejar en 'bounds', si queremos ser
    # rigurosos en esta función, podemos retornarla, aunque el optimizador
    # ya la fuerce en los límites de caja.
    # Se desdobla en dos: FL >= 0 y 1 - FL >= 0
    # Pero para el conteo de desigualdades complejas, nos enfocamos en las acopladas:
    
    # |S3| <= 0.5 * (1 - FL) ---
    # Matemáticamente equivale a: (0.5*(1-FL))^2 - S3^2 >= 0
    c2 = 0.25 * (1 - FL)**2 - S3**2
    
    # S3^2 + 4/9 AFB^2 + S9^2 <= 1/4 (1 - FL)^2 ---
    # Pasamos todo a un lado: RHS - LHS >= 0
    c3 = 0.25 * (1 - FL)**2 - (S3**2 + (4.0/9.0)*AFB**2 + S9**2)
    
    # 4 S4^2 + S7^2 <= FL (1 - FL - 2 S3) ---
    # RHS - LHS >= 0
    c4 = FL * (1 - FL - 2*S3) - (4*S4**2 + S7**2)
    
    # S5^2 + 4 S8^2 <= FL (1 - FL + 2 S3) ---
    # RHS - LHS >= 0
    c5 = FL * (1 - FL + 2*S3) - (S5**2 + 4*S8**2)
    
    return np.array([c2, c3, c4, c5])

def get_profiled_boundary(x_idx, y_idx, n_steps=60):
    
    # 1. PASO NUEVO: Encontrar los límites REALES de X permitidos por la física
    # En lugar de usar rangos fijos, preguntamos al optimizador hasta dónde llega X.
    
    def objective_min_x(p): return p[x_idx]
    def objective_max_x(p): return -p[x_idx]
    def phys_constraints(p): return constraints(p)
    
    cons = [{'type': 'ineq', 'fun': phys_constraints}]
    bounds_opt = [(0, 1)] + [(-1, 1)]*7
    
    # Semilla central segura
    x0 = np.array([0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    
    # Buscamos X_min real
    res_xmin = minimize(objective_min_x, x0, method='SLSQP', bounds=bounds_opt, constraints=cons, tol=1e-6)
    real_start = res_xmin.fun if res_xmin.success else 0.0
    
    # Buscamos X_max real
    res_xmax = minimize(objective_max_x, x0, method='SLSQP', bounds=bounds_opt, constraints=cons, tol=1e-6)
    real_end = -res_xmax.fun if res_xmax.success else 1.0
    
    # ---------------------------------------------------------
    # 2. Ahora escaneamos EXACTAMENTE entre esos límites encontrados
    # Usamos un margen epsilon mucho más pequeño, solo para estabilidad
    epsilon = 1e-5 
    x_vals = np.linspace(real_start + epsilon, real_end - epsilon, n_steps)
    
    y_min_vals = []
    y_max_vals = []
    valid_x = []
    bounds = [(0, 1)] + [(-1, 1)]*7
    # Actual (puede fallar en regiones complejas)
    base_guesses = [...]

    # # Sugerencia: Añadir semillas en extremos de parámetros
    # base_guesses = [
    #     # Centro
    #     np.array([0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     # Extremos de FL
    #     np.array([0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     np.array([0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     # Combinaciones de signos para parámetros acoplados
    #     np.array([0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
    #     np.array([0.3, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]),
    #     # Punto con FL=0.5, S3 máximo/minimo permitido
    #     np.array([0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     np.array([0.5, -0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    # ]
    # --- SEMILLAS ---
    base_guesses = [
        np.array([0.5,  0.0, 0,0,0,0,0,0]), # Centro
        np.array([0.5,  0.2, 0,0,0,0,0,0]), # S3 positivo
        np.array([0.5, -0.2, 0,0,0,0,0,0]), # S3 negativo
        # Semilla para FL -> 0 (Arregla los cortes en V de S9 vs S3)
        np.array([0.01, 0.0, 0,0,0,0,0,0]), 
        # Semilla para FL -> 1 (Arregla la punta del triángulo AFB vs FL)
        np.array([0.99, 0.0, 0,0,0,0,0,0]), 
    ]

    for x_val in x_vals:
        def objective_min(p): return p[y_idx]
        def objective_max(p): return -p[y_idx]
        def fix_x_constraint(p): return p[x_idx] - x_val
        def phys_constraints(p): return constraints(p)
        cons = [{'type': 'eq', 'fun': fix_x_constraint},{'type': 'ineq', 'fun': phys_constraints}]
        best_min = np.inf
        best_max = -np.inf
        found_valid_min = False
        found_valid_max = False
        
        for seed in base_guesses:
            x0 = seed.copy()
            x0[x_idx] = x_val 
            
            # --- Buscar MÁXIMO ---
            res_max = minimize(objective_max, x0, method='SLSQP', bounds=bounds, constraints=cons, tol=1e-4)
            if res_max.success and np.min(constraints(res_max.x)) > -1e-5:
                val = -res_max.fun
                if val > best_max: 
                    best_max = val
                    found_valid_max = True

            # --- Buscar MÍNIMO ---
            res_min = minimize(objective_min, x0, method='SLSQP', bounds=bounds, constraints=cons, tol=1e-4)
            if res_min.success and np.min(constraints(res_min.x)) > -1e-5:
                val = res_min.fun
                if val < best_min: 
                    best_min = val
                    found_valid_min = True

        if found_valid_min and found_valid_max and best_max >= best_min:
            valid_x.append(x_val)
            y_max_vals.append(best_max)
            y_min_vals.append(best_min)
    
    return np.array(valid_x), np.array(y_min_vals), np.array(y_max_vals)

In [ ]:
import numpy as np
import os
import time
from itertools import combinations

def save_boundaries(n_steps=150):
    params = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
    pairs = list(combinations(enumerate(params), 2))
    
    # Crear directorios
    os.makedirs("Plots/BoundaryData", exist_ok=True)
    
    boundary_results = {}
    print(f"Iniciando cálculo de {len(pairs)} fronteras...")
    
    for i, ((idx_x, name_x), (idx_y, name_y)) in enumerate(pairs):
        start = time.time()
        vx, v_min, v_max = get_profiled_boundary(idx_x, idx_y, n_steps=n_steps)
        
        # Guardamos en un diccionario usando un string único como llave
        key = f"{name_x}_vs_{name_y}"
        boundary_results[key] = {'x': vx, 'min': v_min, 'max': v_max}
        
        elapsed = time.time() - start
        print(f"[{i+1}/28] {key} calculado en {elapsed:.2f}s")

    # Guardar todo en un solo archivo comprimido
    np.savez_compressed("Plots/BoundaryData/theoretical_boundaries.npz", **boundary_results)
    print("\n¡Datos de fronteras guardados en Plots/BoundaryData/theoretical_boundaries.npz!")

save_boundaries(n_steps=200)

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
import pandas as pd
from itertools import combinations
import os
hep.style.use("CMS")

def plot_from_saved_data(df_data):
    data = np.load("Plots/BoundaryData/theoretical_boundaries.npz", allow_pickle=True)
    params = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
    latex_labels = {'FL': r'$F_L$', 'S3': r'$S_3$', 'S9': r'$S_9$', 'AFB': r'$A_{FB}$', 'S4': r'$S_4$', 'S7': r'$S_7$', 'S5': r'$S_5$', 'S8': r'$S_8$'}
    pairs = list(combinations(params, 2))
    
    output_dir = "Plots/BoundaryData"
    os.makedirs(output_dir, exist_ok=True)    
    print("Generando ploteos con estilo CMS...")

    for name_x, name_y in pairs:
        key = f"{name_x}_vs_{name_y}"
        if key not in data: continue
        res = data[key].item()
        vx, v_min, v_max = res['x'], res['min'], res['max']
        

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.scatter(df_data[name_x], df_data[name_y], s=1, alpha=1, c='b', label='Monte Carlo', rasterized=True, marker='o', linewidths=0)   
        ax.plot(vx, v_max, color='red', linestyle='--', lw=1, label='Theoretical Limit')
        ax.plot(vx, v_min, color='red', linestyle='--', lw=1)

        ax.set_xlabel(latex_labels[name_x], fontsize=12)
        ax.set_ylabel(latex_labels[name_y], fontsize=12)
        ax.grid(True, linestyle=':', alpha=0.4)
        if name_x == 'FL': ax.set_xlim(0, 1)
        if name_y == 'FL': ax.set_ylim(0, 1)
        
        ax.legend(loc='best', fontsize=12, frameon=True, framealpha=0.8, edgecolor='white')
        ax.tick_params(axis='both', which='major', labelsize=20, length=10, width=1)
        ax.tick_params(axis='both', which='minor', length=5, width=1)

        hep.cms.label(data=False, rlabel="Theoretical Phase Space", loc=0, ax=ax)

        plt.tight_layout()
        plt.savefig(f"{output_dir}/{key}_CMS.png", dpi=150, bbox_inches='tight')
        plt.close(fig)

    print(f"Hecho. Gráficas CMS guardadas en: {output_dir}")

# Ejecución
if 'df_phys_region' in locals():
    plot_from_saved_data(df_phys_region)

# Proyecciones V2 

In [13]:
import zfit
from zfit import z
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# --- REDEFINICIÓN: Eliminamos _integrate para permitir proyecciones numéricas ---
class FullAngular_Physical_PDF(zfit.pdf.BasePDF):
    def __init__(self, obs, FL, S3, S9, AFB, S4, S7, S5, S8, name="FullAngular_Physical_PDF"):
        params = {
            'FL': FL, 'S3': S3, 'S9': S9, 'AFB': AFB,
            'S4': S4, 'S7': S7, 'S5': S5, 'S8': S8
        }
        super().__init__(obs, params, name=name)
    
    def _unnormalized_pdf(self, x):
        vars_list = z.unstack_x(x)
        cos_l = vars_list[0]
        cos_k = vars_list[1]
        phi   = vars_list[2]
        
        sin_k = tf.sqrt(1.0 - cos_k**2)
        sin_l = tf.sqrt(1.0 - cos_l**2)
        sin2_k = sin_k**2
        cos2_k = cos_k**2
        sin2_l = sin_l**2
        
        cos2l_term = 2.0 * cos_l**2 - 1.0
        sin2l_term = 2.0 * sin_l * cos_l
        sin2k_term = 2.0 * sin_k * cos_k
        
        cos_phi = tf.cos(phi)
        sin_phi = tf.sin(phi)
        cos2_phi = tf.cos(2.0 * phi)
        sin2_phi = tf.sin(2.0 * phi)

        FL = self.params['FL']
        S3 = self.params['S3']
        S9 = self.params['S9']
        AFB = self.params['AFB']
        S4 = self.params['S4']
        S7 = self.params['S7']
        S5 = self.params['S5']
        S8 = self.params['S8']
        
        term1 = 0.75 * (1.0 - FL) * sin2_k
        term2 = FL * cos2_k
        term3 = 0.25 * (1.0 - FL) * sin2_k * cos2l_term
        term4 = -1.0 * FL * cos2_k * cos2l_term
        term5 = S3 * sin2_k * sin2_l * cos2_phi
        term6 = S4 * sin2k_term * sin2l_term * cos_phi
        term7 = S5 * sin2k_term * sin_l * cos_phi
        term8 = (4.0/3.0) * AFB * sin2_k * cos_l
        term9 = S7 * sin2k_term * sin_l * sin_phi
        term10 = S8 * sin2k_term * sin2l_term * sin_phi
        term11 = S9 * sin2_k * sin2_l * sin2_phi
        
        pdf = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8 + term9 + term10 + term11
        return pdf

    # NOTA: Se ha eliminado _integrate explícitamente.
    # zfit ahora usará integración numérica por defecto, lo que permite
    # calcular proyecciones (integrales parciales) correctamente.

In [14]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import zfit
import numpy as np
import os
import tensorflow as tf
from scipy.stats import chi2
import mplhep as hep # Asegúrate de tener instalado mplhep (pip install mplhep)

# Configuración de estilo CMS si lo deseas (opcional)
# hep.style.use("CMS")

def plot_analytical_projections_cms(sampler, phys_params_dict, folder_out="Plots/Analytic_Projections_CMS", n_bins=30):
    """
    Genera proyecciones con estilo CMS:
    - Panel superior: Datos vs Fit + Caja de texto con parámetros y Chi2.
    - Panel inferior: Pulls (Residuales normalizados).
    """
    if not os.path.exists(folder_out):
        os.makedirs(folder_out)

    obs_space = sampler.space
    observables = obs_space.obs  
    
    # 1. Instanciar la PDF Física con los parámetros del Best Fit
    print(f">>> Instanciando PDF física para proyecciones...")
    pdf_model = FullAngular_Physical_PDF(
        obs=obs_space,
        **phys_params_dict 
    )

    # Etiquetas LaTeX para los ejes
    latex_labels = {
        observables[0]: r'$\cos(\theta_l)$',
        observables[1]: r'$\cos(\theta_K)$',
        observables[2]: r'$\phi$'
    }

    # Total de eventos (para escalar la PDF)
    n_total = sampler.n_events.numpy()

    for obs_name in observables:
        print(f"--- Procesando plot CMS para: {obs_name} ---")
        
        # A. Crear la PDF Proyectada (integrando las otras variables)
        vars_to_integrate = [var for var in observables if var != obs_name]
        limits_integration = obs_space.with_obs(vars_to_integrate)
        pdf_proj = pdf_model.create_projection_pdf(limits=limits_integration)
        
        # B. Preparar Datos (Histograma)
        idx = observables.index(obs_name)
        data_column = sampler.unstack_x()[idx] # Corrección de extracción de lista
        
        if hasattr(data_column, "numpy"):
            data_np = data_column.numpy()
        else:
            data_np = np.array(data_column)
            
        counts, bin_edges = np.histogram(data_np, bins=n_bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        bin_width = bin_edges[1] - bin_edges[0]
        y_err_data = np.sqrt(counts)
        
        # C. Calcular Modelo (Curva Suave y Puntos para Chi2)
        lower, upper = obs_space.with_obs(obs_name).limit1d
        
        # C.1 Para el plot suave
        x_fine = np.linspace(lower, upper, 400)
        y_smooth_density = pdf_proj.pdf(x_fine).numpy()
        y_smooth_counts = y_smooth_density * n_total * bin_width
        
        # C.2 Para Chi2 y Pulls (evaluado en el centro del bin)
        y_model_at_centers_density = pdf_proj.pdf(bin_centers).numpy()
        y_model_at_centers = y_model_at_centers_density * n_total * bin_width
        
        # D. Calcular Pulls y Chi2
        # Pull = (Data - Model) / Error_Data
        # Evitamos división por cero donde counts es 0
        mask = counts > 0
        pulls = np.zeros_like(counts, dtype=float)
        pulls[mask] = (counts[mask] - y_model_at_centers[mask]) / y_err_data[mask]
        
        chi2_val = np.sum(pulls[mask]**2)
        ndof = n_bins - 1 # Aproximación para proyección 1D
        chi2_red = chi2_val / ndof
        p_val = 1 - chi2.cdf(chi2_val, ndof)

        # ====== PLOTTING ESTILO CMS ======
        fig = plt.figure(figsize=(9, 9))
        gs = GridSpec(2, 1, height_ratios=[3.5, 1], hspace=0.08)
        ax0 = fig.add_subplot(gs[0])
        ax1 = fig.add_subplot(gs[1])

        # --- Panel Superior (Datos + Modelo) ---
        ax0.errorbar(bin_centers, counts, yerr=y_err_data, xerr=bin_width/2.0, 
                     fmt='ks', markersize=4, elinewidth=1, capsize=2, label='Toy (data)')

        ax0.plot(x_fine, y_smooth_counts, '-', linewidth=2, label='Analytical PDF', color='b')
        
        # Límites dinámicos con margen superior
        max_data = np.max(counts + y_err_data)
        max_model = np.max(y_smooth_counts)
        y_max_plot = max(max_data, max_model)
        ax0.set_ylim(0, y_max_plot * 1.5)
        
        ax0.set_xlim(lower, upper)
        ax0.set_xticklabels([]) # Ocultar etiquetas X del panel superior
        ax0.set_ylabel(f'Events / {bin_width:.2f}', fontsize=16)
        ax0.grid(True, alpha=0.2)
        ax0.legend(loc='upper right', fontsize=13, frameon=False, bbox_to_anchor=(0.98, 0.88))

        # Etiqueta CMS
        # Nota: Si falla hep.cms.label, comenta esta línea.
        try:
            hep.cms.label(data=False, label="Simulation", loc=0, ax=ax0, rlabel="13 TeV")
        except:
            ax0.text(0.05, 1.01, "CMS Simulation (Preliminary)", transform=ax0.transAxes, fontsize=14, fontweight='bold')

        # --- Caja de Texto con Estadísticas ---
        # Extraemos valores del diccionario
        FL_val = phys_params_dict.get('FL', 0)
        AFB_val = phys_params_dict.get('AFB', 0)
        S3_val = phys_params_dict.get('S3', 0)
        S9_val = phys_params_dict.get('S9', 0)

        param_text = (
            rf'$F_L = {FL_val:.3f}$' + '\n' +
            rf'$A_{{FB}} = {AFB_val:.3f}$' + '\n' +
            rf'$S_3 = {S3_val:.3f}$' + '\n' +
            rf'$S_9 = {S9_val:.3f}$' + '\n' +
            r'----------------' + '\n' + 
            rf'$\chi^2/\mathrm{{ndof}} = {chi2_val:.1f}/{ndof} = {chi2_red:.2f}$' + '\n' +
            rf'$p$-value = {p_val:.3f}'
        )

        ax0.text(0.05, 0.92, param_text, transform=ax0.transAxes, fontsize=12, 
                 verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='#dddddd'))

        # --- Panel Inferior (Pulls) ---
        ax1.errorbar(bin_centers, pulls, 
                     yerr=1.0,           
                     xerr=0,             
                     fmt='ks',           
                     markersize=3,       
                     elinewidth=1.0,     
                     capsize=0)          
        
        # Líneas de referencia
        ax1.axhline(0, color='black', linewidth=1.0, linestyle='-')
        ax1.axhline(3, color='gray', linestyle=':', linewidth=1, alpha=0.8) # 3 sigma
        ax1.axhline(-3, color='gray', linestyle=':', linewidth=1, alpha=0.8)
        
        # Relleno de bandas sigma (opcional, estilo ROOT)
        ax1.fill_between([lower, upper], -1, 1, color='yellow', alpha=0.1) # 1 sigma
        ax1.fill_between([lower, upper], -2, 2, color='green', alpha=0.05) # 2 sigma

        xlabel = latex_labels.get(obs_name, obs_name)
        ax1.set_xlabel(xlabel, fontsize=16)
        ax1.set_ylabel(r'Pull $(\sigma)$', fontsize=13)
        ax1.set_xlim(lower, upper)
        ax1.set_ylim(-4.9, 4.9)
        ax1.grid(True, alpha=0.2)

        # Ajuste de ticks
        ax0.tick_params(axis='both', which='major', labelsize=14) 
        ax1.tick_params(axis='both', which='major', labelsize=14)

        # Guardar
        outname = os.path.join(folder_out, f"CMS_Proj_{obs_name}.png")
        plt.savefig(outname, bbox_inches='tight', dpi=200)
        plt.close(fig)
        print(f"    -> Gráfica guardada: {outname}")

    print(">>> Todas las proyecciones estilo CMS generadas.")

# --- LLAMADA A LA FUNCIÓN ---
plot_analytical_projections_cms(
    sampler=sampler,
    phys_params_dict=best_fit_phys_dict,
    folder_out="Plots/Analytic_Projections_CMS",
    n_bins=30
)

>>> Instanciando PDF física para proyecciones...
--- Procesando plot CMS para: cosThetaL ---
Estimated integral error ( 4.7820036572242623e-06 ) larger than tolerance ( 3e-06 ), which is maybe not enough (but maybe it's also fine). You can (best solution) implement an anatytical integral (see examples in repo) or manually set a higher number on the PDF with 'update_integration_options' and increase the 'max_draws' (or adjust 'tol'). If partial integration is chosen, this can lead to large memory consumption.This is a new warning checking the integral accuracy. It may warns too often as it is Work In Progress. If you have any observation on it, please tell us about it: https://github.com/zfit/zfit/issues/new/chooseTo suppress this warning, use zfit.settings.set_verbosity(-1).
Estimated integral error ( 2.3815317593743481e-05 ) larger than tolerance ( 3e-06 ), which is maybe not enough (but maybe it's also fine). You can (best solution) implement an anatytical integral (see examples in r